# Intro to Enums

> Andrew Kubera

> COhPy - Feb 26, 2018

### What is an Enumerated Type?

* Simply a type comprising a finite-set of named values

* Standard example is a card suit:
```
    Hearts, Spades, Diamonds, Clubs
```

* Most languages store these internally as as simple integers
  * Compilers often typecheck, providing some safety
  * Easily copiable 

Enumerated types are a common tool in programming languages:

* C-based
* Java
* Rust
* Swift
* TypeScript
* Haskell
* Lisp

- Missing : JavaScript, Ruby, *Python???*
  - Unnecessary in dynamically-typed languages?

`enum` module added to Python standard library in version *3.4*

Here's how to use it:

In [ ]:
from enum import Enum

In [ ]:
class A(Enum):
    X = 1
    Y = 2
    Z = 3

Access members as usual:

In [ ]:
A.X

In [ ]:
A.X == 1

In [ ]:
A.X == A.X

In [ ]:
A.Y == A.X

In [ ]:
a = A.X
b = A.X

a is b

Values may be looked up by "name" using `[ ]` notation

In [ ]:
key = "Y"

...

A[key]

Enums have dict-like access!

In [ ]:
A[key] is A.Y

Also supports "backwards" lookup via `( )`:

In [ ]:
A(1)

(note: this would *usually* be creating a new object of type `A`)

**Bidirectional mapping** between a name (string) and some value!

### "Standard" errors when doing lookups

In [ ]:
A.W  # AttributeError

In [ ]:
A['W']  # KeyError

In [ ]:
A[1]  # KeyError

In [ ]:
A(9)  # ValueError

## Iteration over members

Enums are *iterable* over all values (order is preserved)

In [ ]:
list(A)

### Alternative means of construction

In [ ]:
B = Enum("B", "X Y Z")

In [ ]:
B.X

In [ ]:
B(3)

In [ ]:
B = Enum("B", [('X', 90), ('Y', 45)])

In [ ]:
B.X

### Lets look more at the values...

In [ ]:
# Values can be mixed types
class B(Enum):
    X = 1
    Y = "some string."
    Z = 0.3
    L = []  # note the list!

In [ ]:
B("some string.")

In [ ]:
B([])

In [ ]:
B(0.3)

<small>BTW, you should **NOT** use floating points to do indexing</small>

In [ ]:
key = 0.1 + 0.2

In [ ]:
B(key)  # B(0.3)

# Namedtuple like behavior

In [ ]:
# Cannot reassign values
B.X = 2

In [ ]:
B.L.append(1)

We will have to start thinking differently.....

## Look deeper at the values

In [ ]:
class A(Enum):
    X = 1
    Y = 2
    Z = 3

In [ ]:
A.X

In [ ]:
isinstance(A.X, int)

In [ ]:
isinstance(A.X, A)

In [ ]:
type(A.X)

When creating the class:

* The members are "intercepted" and wrapped with instances of the class.
* After that, the class is "frozen" and *no further instances* may be created

One little "goof":

In [ ]:
A.X.X.Y

^ Because instances of objects have access to members of their class, all enums have each-other as attributes (Don't use this!)

So where are `1, 2, 3` in A?

In [ ]:
A.X.value

In [ ]:
A.X.name

The `value` & `name` attributes are ***the*** important things to know about python Enums 

In [ ]:
B.L

In [ ]:
B.L.value.append(7)

In [ ]:
B.L

So they are *semi-*immutable (like tuples)

### Methods

In [ ]:
class A(Enum):
    X = 1
    Y = 2
    
    def add_ten(self):
        return self.value + 10

In [ ]:
list(A)

`add_ten` **not** included as enum member -- behaves as typical python method

In [ ]:
A.add_ten

In [ ]:
A.X.add_ten()

In [ ]:
class C(Enum):
    X = 1
    Y = lambda a: print('Y(a=%r)' % a)

In [ ]:
list(C)

In [ ]:
C.X.Y() 

So lambdas are interpreted as methods (as usual)

Can I get a little crazy now?

In [ ]:
class A(Enum):
    
    class J:
        def __init__(self):
            print("constructing J")
            
    class M:
        def __init__(self):
            print("constructing M")


In [ ]:
list(A)

In [ ]:
m = A.M.value()

In [ ]:
type(m)

In [ ]:
class A(Enum):
    
    class J:
        def __init__(self):
            print("constructing J")
            
    class M:
        def __init__(self):
            print("constructing M")

    def New(self):
        return self.value()
    
    @classmethod
    def From(cls, foo):
        return cls[foo.upper()].New()

In [ ]:
classname = 'J'
A[classname].New()

In [ ]:
A.From('m')

Back to basics...

## Cannot subclass Enums

In [ ]:
class A(Enum):
    X = 1
    Y = 2

In [ ]:
# TypeError: Cannot extend enumerations
class B(A):
    Z = 3

In [ ]:
[(a.name, a.value) for a in A] 

In [ ]:
B = Enum("B", [(a.name, a.value) for a in A] + [('Z', 3)])
list(B)

## Other Enums

### IntEnum

* The values must be integers
    * Or rather, they are `int()` compatible
* The type inherits from `int`, so comparisons works 
    * no need for `.value`

In [ ]:
from enum import IntEnum

In [ ]:
class A(IntEnum):
    X = 1
    Y = 2

In [ ]:
A.X

In [ ]:
A.X == 1

In [ ]:
class B(IntEnum):
    X = 1
    Y = '2'

In [ ]:
B.Y == 2

In [ ]:
B('2')

### Flag & IntFlag

* New in py3.6
* Flags support bitwise `&`, `|` operations to *combine* enumerated values
* Bitflags should be kept to powers of two

In [ ]:
from enum import Flag, IntFlag

In [ ]:
class A(Flag):
    X = 1  # 1 << 0
    Y = 2  # 1 << 1
    Z = 4  # 1 << 2

In [ ]:
a = A.X | A.Z
a

In [ ]:
isinstance(a, A)

In [ ]:
a & A.X

In [ ]:
a & A.Y

In [ ]:
a & A.Z

auto-enumeration works as expected

In [ ]:
A = IntFlag("A", "W X Y Z")
list(A)

In [229]:
A(13)

<A.Z|Y|W: 13>

In [ ]:
for a in A:
    print(f'{a.name} = 0b{a:05b}')

### When to use?

* When you have finite set of things and want name & value lookup + iteration for free
    * "Finite" means "Nameable"

* Examples:
    * Finite states:
        * On/Off
        * Low, Medium, High
    * String or Byte "validation" (IntFlag)

## Example: Suit Of Cards

In [ ]:
class Suit(Enum):
    Hearts = 1
    Clubs = 2
    Spades = 3
    Diamonds = 4
    
    def is_red(self):
        return self in (Suit.Hearts, Suit.Diamonds)

In [ ]:
list(Suit)

In [ ]:
Suit.Diamonds.is_red()

In [ ]:
from itertools import product
deck = list(product(Suit, range(1, 14)))

In [ ]:
deck[:5]

In [ ]:
from random import shuffle
shuffle(deck)
deck[:6]

In [ ]:
class Card:
    def __init__(self, suit, face):
        if isinstance(suit, str):
            suit = Suit[suit]
        self.suit = suit
        self.face = face

    def __repr__(self):
        if 1 < self.face < 11:
            f = self.face
        else:
            f = self.FaceToLetter(self.face).name
        return "<Card %s %s>" % (f, self.suit.name)

    class FaceToLetter(Enum):
        A = 1
        J = 11
        Q = 12
        K = 13

In [ ]:
from itertools import starmap
deck = list(starmap(Card, product(Suit, range(1, 14))))

In [ ]:
deck[::5]

## Example: Unix File Permissions

In [219]:
class Perm(IntFlag):
    READ = 1
    WRITE = 2
    EXEC = 4
    
    def __str__(self):
        return (('r' if self & self.READ else '-') +
                ('w' if self & self.WRITE else '-') +
                ('x' if self & self.EXEC else '-'))

In [220]:
'%s' % (Perm.READ | Perm.EXEC)

'r-x'

In [226]:
Perm(6)

<Perm.EXEC|WRITE: 6>

* Full UNIX file permission has a "Perm" for `user`, `group`, `all`.
* NOT a good case for Enum
    - Many (3 × 2^3) states to choose from
    - You do not want to name them all!

In [232]:
class UnixPermission:
    def __init__(self, user: Perm, group: Perm, all: Perm):
        self.user = user
        self.group = group
        self.all = all
    
    @classmethod
    def from_int(cls, num):
        return cls(Perm((num >> 6) & 7),
                   Perm((num >> 3) & 7),
                   Perm((num >> 0) & 7))

    def __str__(self):
        return '%s%s%s' % (self.user, self.group, self.all)

In [235]:
str(UnixPermission.from_int(0o755))

'rwxr-xr-x'

## Example : HTTP Methods & Status Codes


* [HTTP Methods](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Summary_table)
* [HTTP Status Codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)


Famous Status Codes:
    * 200 - OK
    * 404 - Not Found
    * 500 - Internal Server Error

In [ ]:
HttpMethod = Enum("HttpMethod", "GET POST PUT DELETE")

In [ ]:
try:
    method = HttpMethod[method_str] 
except KeyError:
    raise UnknownHttpMethodException(method_str)

In [ ]:
class HttpStatus(IntEnum):
    OK = 200
    NOT_FOUND = 404
    INTERNAL_SERVER_ERROR = 500

In [ ]:
class HttpError(Exception):
    pass

class HttpErrorNotFound(HttpError):
    status = HttpStatus.NOT_FOUND

class HttpErrorISE(HttpError):
    status = HttpStatus.INTERNAL_SERVER_ERROR


In [ ]:
from http import HTTPStatus
HTTPStatus

In [ ]:
HTTPStatus['OK']

In [ ]:
HTTPStatus["OK"].description

In [ ]:
HTTPStatus(404)

In [ ]:
list(HTTPStatus)

## Summary

* Enums are a useful tool in your programming toolbox
  * Should be aware of them and when to use them
* Nothing truely new (you could implement with dictionaries), but get automatic features
* For more information, check the documentation https://docs.python.org/3/library/enum.html


# END